# 

# 2_ReviseCareAI_Reviews_Enriched

**Purpose of that notebook**: extract on a daily basis the last 5 reviews from the preconfigured hospitals and put them into the enriched layer of table.

In [11]:
# Copy the output to the processing notebook as a first cell
# Remove all unnecesary hospitals
# ******
hospitals_config = '''
Zhugczc Fslun Pudljfbo Bmn Wfbyx:ChIJG1uXAtpU2YARlrrtB9mAqjw
Nhkqg Qxmhwcni Nsadjzmy:ChIJ8Q0HQF9V2YAR9clz3L9QTHw
Spbiuxp Nibrgksx Opn Bkuis:ChIJb-g3WeFU2YARsXx7bJpLPpA
Ozlvgrmsk Oezdmqc Mdeygl xp ME Slv Yhtbw Mxjzfm:ChIJ19C1PtZU2YARoB1ec6RBNoA
Eryjl Hwxxqiui fp Wkb Kesch:ChIJjYO8adpU2YAR4I_h-DE6NH0
Bebcb Yjlpeel Igxzqm:ChIJK1gnL6dU2YARVmtye1FMG7w
Fsoqkz Gbblenlrq Gxxvgnxz - Vkl Tuzpx:ChIJDUfwvx1V2YARkzwov5fQ1es
Hcxceu Pentydidcp Qbv Skvrh Ksfcxiw Ewzsgv:ChIJASS3IEv-24ARWc3gkB-I7cA
Jpog Ardxfyzf’m Lunmipva - Emi Zwxov:ChIJSZqBu9dU2YARhhpz0EGLooo
Voteatw Fbmbdvzj Bcpxcqvp Ym Hwfdb:ChIJK1n7it4G3IARtIq4C6lYJ2I
Dheul Agvv Lbiws Ajbhtbxr kwk Hjjps & Npxusmgf:ChIJ99-RUl9T2YARIYBo3w54zF8
Nazlxrho Sksyag Arkuhfsv:ChIJQy5TnylS2YARqzvb9ZDWsVg
Qdmpgqi Wtwrg Wulsduqv Hlfdm Icqxt:ChIJg-Mx4uNN2YAR5Vl8xGWVhPU
Rfbml Rdxnqjibw Fjcdydgu:ChIJ-718aG1X2YARWrmlhhk4aJA
Lyekik Gqdjoqdhqh Gdmx Pnirimg Gixmbm:ChIJr80_QsJV2YARkUg6KqwSUuo
UCOB IZCLLINJB:ChIJk3jlG8VV2YARtD8bzFNhHzY
NO Unu Dzecd Scpzkvqi js Anfgbyaq Cgdpzcvd:ChIJm1IklZlw44kRENEZvJgIuBc
Nwnaiy Edlrban Jedmyz db FO Fji Lmsji Ffmuls:ChIJb9IXHdoG3IARYVL4U5OQ7y8
Acari Krwbtsfk Ljefbzrk:ChIJDQfFKEpT2YARAWp6gvF8NyU
MN Fiwloru Tytnja-Tui Rfksm:ChIJy6yiHdAG3IAR97uNw1l9xE4
'''

StatementMeta(, b6972da7-481a-4427-8d2f-e4939c987f2c, 23, Finished, Available, Finished)

In [12]:
%run 0_ReviseCareAI_Utils

StatementMeta(, b6972da7-481a-4427-8d2f-e4939c987f2c, 32, Finished, Available, Finished)

Variables GMAP_API_KEY, OPENAI_API_KEY, AZURE_OPEN_AI_KEY, AZURE_OPEN_AI_ENDPOINT were initialized
Google client gmaps was is initialized
Function list_to_dataframe() is initialized
Function get_ids_from_config() is initialized
Function get_hospital_details() is initialized
Function openai_get_answer() is initialized
UDF Function openai_udf() is initialized
Function string_masking() is initialized
UDF Function string_masking_udf() is initialized


In [3]:
place_ids = get_ids_from_config(hospitals_config)
print("The following placess will be processed: ", place_ids)

StatementMeta(, b6972da7-481a-4427-8d2f-e4939c987f2c, 15, Finished, Available, Finished)

The following placess will be processed:  ['ChIJG1uXAtpU2YARlrrtB9mAqjw', 'ChIJ8Q0HQF9V2YAR9clz3L9QTHw', 'ChIJb-g3WeFU2YARsXx7bJpLPpA', 'ChIJ19C1PtZU2YARoB1ec6RBNoA', 'ChIJjYO8adpU2YAR4I_h-DE6NH0', 'ChIJK1gnL6dU2YARVmtye1FMG7w', 'ChIJDUfwvx1V2YARkzwov5fQ1es', 'ChIJASS3IEv-24ARWc3gkB-I7cA', 'ChIJSZqBu9dU2YARhhpz0EGLooo', 'ChIJK1n7it4G3IARtIq4C6lYJ2I', 'ChIJ99-RUl9T2YARIYBo3w54zF8', 'ChIJQy5TnylS2YARqzvb9ZDWsVg', 'ChIJg-Mx4uNN2YAR5Vl8xGWVhPU', 'ChIJ-718aG1X2YARWrmlhhk4aJA', 'ChIJr80_QsJV2YARkUg6KqwSUuo', 'ChIJk3jlG8VV2YARtD8bzFNhHzY', 'ChIJm1IklZlw44kRENEZvJgIuBc', 'ChIJb9IXHdoG3IARYVL4U5OQ7y8', 'ChIJDQfFKEpT2YARAWp6gvF8NyU', 'ChIJy6yiHdAG3IAR97uNw1l9xE4']


In [27]:
all_reviews_df = None

for i in place_ids[0:21]:
    details = get_hospital_details(place_id = i)
    append_df = list_to_dataframe(data = [details])
    
    if all_reviews_df is None:
        all_reviews_df = append_df
    else:
        all_reviews_df = all_reviews_df.unionAll(append_df)
    

masked_all_reviews_df = (
    all_reviews_df
    .withColumn("formatted_address_tmp", string_masking_udf(all_reviews_df["formatted_address"])).drop("formatted_address").withColumnRenamed("formatted_address_tmp", "formatted_address")
    .withColumn("name_tmp", string_masking_udf(all_reviews_df["name"])).drop("name").withColumnRenamed("name_tmp", "name")
    .withColumn("place_id_tmp", string_masking_udf(all_reviews_df["place_id"])).drop("place_id").withColumnRenamed("place_id_tmp", "place_id")
)
display(masked_all_reviews_df)

StatementMeta(, b6972da7-481a-4427-8d2f-e4939c987f2c, 47, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0e801d39-0a48-4f77-b0fb-d314a59af289)

In [32]:
from pyspark.sql import Window
from pyspark.sql.functions import explode, col, current_date, from_unixtime, date_sub, row_number
import pyspark.sql.functions as F

all_reviews_exploded_df = (
    all_reviews_df
    .withColumn("processing_date", current_date())
    .withColumn("review_details", explode(col("reviews")))
    .drop("reviews")
    .select(
            col('processing_date').alias('processing_date'),
            col('business_status').alias('hospital_status'),
            string_masking_udf(col('formatted_address')).alias('hospital_address'),
            string_masking_udf(col('name')).alias('hospital_name'),
            string_masking_udf(col('place_id')).alias('hospital_place_id'),
            col('rating').alias('hospital_avg_rating'),
            col('user_ratings_total').alias('hospital_rating_cnt'),
            string_masking_udf(col('review_details.author_name')).alias('review_author'),
            col('review_details.rating').alias('review_rating'),
            col('review_details.text').alias('review_text'),
            from_unixtime(col('review_details.time')).alias('review_time'),
    )    
)

display(all_reviews_exploded_df)

StatementMeta(, b6972da7-481a-4427-8d2f-e4939c987f2c, 52, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d04bfbc3-95c3-46e7-9041-72f3104e19e9)

# Writing to enriched_daily_reviews

In [33]:
processing_date = str(all_reviews_exploded_df.agg(F.max("processing_date")).collect()[0][0])

(
all_reviews_exploded_df.write
.format("delta")
.mode("overwrite")
.option("replaceWhere", f"processing_date = '{processing_date}'")
.save("Files/enriched_daily_reviews")
)

StatementMeta(, b6972da7-481a-4427-8d2f-e4939c987f2c, 53, Finished, Available, Finished)

In [34]:
daily_reviews_df = spark.read.format("delta").load("Files/enriched_daily_reviews")
display(daily_reviews_df)

StatementMeta(, b6972da7-481a-4427-8d2f-e4939c987f2c, 54, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6b094c1e-28f5-40d5-8073-b8b424956206)

# Deduplication to "enriched_final_reviews"

In [35]:
window_spec = Window.partitionBy("hospital_place_id", "review_author_url").orderBy(col("processing_date").desc())
df_with_row_number = daily_reviews_df.withColumn("row_num", row_number().over(window_spec))
dedup_reviews_df = df_with_row_number.filter(col("row_num") == 1).drop("row_num")

display(dedup_reviews_df)

StatementMeta(, b6972da7-481a-4427-8d2f-e4939c987f2c, 55, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b6f5ecd9-a621-43e7-a8a4-16d68464c007)

In [36]:
(
dedup_reviews_df.write
.format("delta")
.mode("overwrite")
.save("Files/enriched_final_reviews")
)

enriched_final_reviews_df = spark.read.format("delta").load("Files/enriched_final_reviews")
display(enriched_final_reviews_df)

StatementMeta(, b6972da7-481a-4427-8d2f-e4939c987f2c, 56, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9faa6b98-b4d0-4123-aeca-1da1069e8f6e)